In [1]:
library(RHOGE)

# example BMI TWAS results from FUSION
data(bmi)
head(bmi)


FILE,ID,CHR,P0,P1,HSQ,BEST.GWAS.ID,BEST.GWAS.Z,EQTL.ID,EQTL.R2,⋯,NSNP,NWGT,MODEL,MODELCV.R2,MODELCV.PV,TWAS.Z,TWAS.P,PERM.PV,PERM.N,PERM.ANL_PV
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,⋯,<int>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>
/u/home/n/nmancuso/pasaniucdata/DATA/FUSION//WEIGHTS//CMC.BRAIN.RNASEQ/CMC.LOC643837.wgt.RDat,LOC643837,1,763000,795000,0.0979,rs17160824,1.96,rs3094315,0.0865,⋯,159,4,lasso,0.084282,1.89e-10,-0.96910,0.332,0,0,0
/u/home/n/nmancuso/pasaniucdata/DATA/FUSION//WEIGHTS//CMC.BRAIN.RNASEQ/CMC.AGRN.wgt.RDat,AGRN,1,956000,991000,0.0655,rs17160824,1.96,rs9442372,0.0521,⋯,200,6,lasso,0.062644,4.15e-08,-0.44390,0.657,0,0,0
/u/home/n/nmancuso/pasaniucdata/DATA/FUSION//WEIGHTS//CMC.BRAIN.RNASEQ/CMC.C1orf159.wgt.RDat,C1orf159,1,1020000,1050000,0.0917,rs17160824,1.96,rs3766191,0.0266,⋯,212,9,enet,0.070706,5.63e-09,-1.51971,0.129,0,0,0
/u/home/n/nmancuso/pasaniucdata/DATA/FUSION//WEIGHTS//CMC.BRAIN.RNASEQ/CMC.SCNN1D.wgt.RDat,SCNN1D,1,1220000,1230000,0.1704,rs9660180,4.89,rs11260570,0.0840,⋯,248,248,bslmm,0.114216,9.28e-14,0.76111,0.447,0,0,0
/u/home/n/nmancuso/pasaniucdata/DATA/FUSION//WEIGHTS//CMC.BRAIN.RNASEQ/CMC.MXRA8.wgt.RDat,MXRA8,1,1290000,1300000,0.0503,rs9660180,4.89,rs1739855,0.0206,⋯,258,7,lasso,0.035945,2.95e-05,-0.17818,0.859,0,0,0
/u/home/n/nmancuso/pasaniucdata/DATA/FUSION//WEIGHTS//CMC.BRAIN.RNASEQ/CMC.LOC148413.wgt.RDat,LOC148413,1,1330000,1340000,0.0648,rs9660180,4.89,rs2649588,0.0071,⋯,270,13,enet,0.058540,1.14e-07,-1.06427,0.287,0,0,0


In [ ]:
# example triglyceride TWAS results from FUSION
data(tg)
head(tg)

In [2]:
# Estimate rho_ge genome-wide for BMI and Triglyerides and approximate sample sizes
ge_cor_res <- rhoge.gw(bmi, tg, 14000, 91000)
head(ge_cor_res)

,RHOGE,SE,TSTAT,DF,P
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.2133573,0.05683608,3.753906,461,0.0001963482


In [3]:
data("grch37.eur.loci")
head(grch37.eur.loci)

CHR,START,STOP
<int>,<int>,<int>
1,10583,1892607
1,1892607,3582736
1,3582736,4380811
1,4380811,5913893
1,5913893,7247335
1,7247335,9365199


In [4]:
library("nadiv")

Loading required package: Matrix



In [5]:
(noBC <- LRTest(full = -2254.148, reduced = -2258.210,
	df = 1, boundaryCorrection = FALSE))

$lambda
[1] 8.124

$Pval
[1] 0.004368307

$corrected.Pval
[1] FALSE

In [6]:
(withBC <- LRTest(full = -2254.148, reduced = -2258.210,
	df = 1, boundaryCorrection = TRUE))
stopifnot(noBC$Pval == 2*withBC$Pval)

$lambda
[1] 8.124

$Pval
[1] 0.002184154

$corrected.Pval
[1] TRUE

In [7]:
  rm(list = ls())
  set.seed(101)
  library(nadiv)
  # create pedigree
  warcolak <- simPedDFC(F = 75, gpn = 4, fsn = 4, s = 2)
  names(warcolak)[1:3] <- c("ID", "Dam", "Sire")
  warcolak$trait2 <- warcolak$trait1 <- NA
  # Define covariance matrices for random effects:
  ## Autosomal additive genetic (trait1)
  Ga_t1 <- matrix(c(0.4, rep(0.399999, 2), 0.4), 2, 2)
  ## Autosomal additive genetic (trait2)
  Ga_t2 <- matrix(c(0.3, rep(0.299999, 2), 0.3), 2, 2)
  ## Sex-chromosomal additive genetic (trait2)
  Gs_t2 <- matrix(c(0.1, rep(0.099999, 2), 0.1), 2, 2)
  ## Autosomal dominance genetic
  Gd <- matrix(c(0.3, rep(0.299999, 2), 0.3), 2, 2)
  ## Environmental (or residual)
  ### Assumes no correlated environmental effects between sexes
  R <- diag(c(0.3, 0.3))

In [9]:

  ## Dominance genetic
  ### trait1 
  tmp_d <- grfx(pedn, G = Gd, incidence = makeD(warcolak[, 1:3], invertD = FALSE)$D)
    var(tmp_d)
  warcolak$t1_d <- tmp_d[cbind(seq(pedn), sexcol)]
  ### trait2
  tmp_d <- grfx(pedn, G = Gd, incidence = makeD(warcolak[, 1:3], invertD = FALSE)$D)
    var(tmp_d)
  warcolak$t2_d <- tmp_d[cbind(seq(pedn), sexcol)]

starting to make D....done 


0.2897524,0.2897498
0.2897498,0.2897492


starting to make D....done 


0.2912346,0.2912446
0.2912446,0.2912566


In [10]:
 ## Residual
  ### trait1
  tmp_r <- grfx(pedn, G = R, incidence = NULL) # warning of identity matrix
    var(tmp_r)
  warcolak$t1_r <- tmp_r[cbind(seq(pedn), sexcol)]
  ### trait2
  tmp_r <- grfx(pedn, G = R, incidence = NULL) # warning of identity matrix
    var(tmp_r)
  warcolak$t2_r <- tmp_r[cbind(seq(pedn), sexcol)]

Warning message in grfx(pedn, G = R, incidence = NULL):
“Incidence matrix used = Identity matrix”


0.2976295867,-0.0008137167
-0.0008137167,0.3003321215


Warning message in grfx(pedn, G = R, incidence = NULL):
“Incidence matrix used = Identity matrix”


0.298459353,-0.001988546
-0.001988546,0.295705748
